## Import Modules

In [1]:
import torch, os
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.autograd import Variable
import torch.autograd as autograd
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torchvision.utils import save_image
import numpy as np
from utils.visdom_utils import VisFunc

## Feature Extractor

In [2]:
class FrontEnd(nn.Module):
    def __init__(self):
        super(FrontEnd, self).__init__()

        self.main = nn.Sequential(
            nn.Conv2d(1,64,4,2,1),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(64,128,4,2,1,bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(128, 1024,7,bias=False),
            nn.BatchNorm2d(1024),
            nn.LeakyReLU(0.1, inplace=True),
        )

    def forward(self,x):
        output = self.main(x)
        return output

## Discriminator D

In [3]:
class Dmodel(nn.Module):
    def __init__(self):
        super(Dmodel, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(1024,1,1),
            nn.Sigmoid()
        )

    def forward(self,x):
        output=self.main(x).view(-1,1)
        return output

## Coder Classifier Q

In [4]:
class Qmodel(nn.Module):
    def __init__(self):
        super(Qmodel,self).__init__()

        self.conv = nn.Conv2d(1024,128,1,bias=False)
        self.bn = nn.BatchNorm2d(128)
        self.lReLU = nn.LeakyReLU(0.1, inplace=True)
        self.conv_disc = nn.Conv2d(128,10,1)
        self.conv_mu = nn.Conv2d(128,2,1)
        self.conv_var = nn.Conv2d(128,2,1)

    def forward(self,x):
        y = self.conv(x)
        disc_logits = self.conv_disc(y).squeeze()
        mu = self.conv_mu(y).squeeze()
        var = self.conv_var(y).squeeze().exp()
        return disc_logits, mu, var


## Generator G

In [5]:
class Gmodel(nn.Module):
    def __init__(self):
        super(Gmodel, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(74, 1024,1,1, bias=False),
            nn.BatchNorm2d(1024),
            nn.ReLU(True),
            nn.ConvTranspose2d(1024, 128,7,1,bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(128,64,4,2,1,bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.ConvTranspose2d(64,1,4,2,1,bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        output = self.main(x)
        return output


## Initialize Models and Optimizer and Data Loader

In [6]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)
    
# Models
FE=FrontEnd()
D=Dmodel()
Q=Qmodel()
G=Gmodel()

for i in [FE, D, Q, G]:
    i.cuda()
    i.apply(weights_init)

# Optimizers
optimD = optim.Adam([{'params':FE.parameters()},
                     {'params':D.parameters()}],
                    lr=0.0002, betas=(0.5, 0.99) )

optimG = optim.Adam([{'params':G.parameters()},
                     {'params':Q.parameters()}],
                    lr=0.001, betas=(0.5, 0.99) )

# Datasets
batch_size = 256

# Train using 10K Test Images
train_data = dset.MNIST('./dataset', train=False, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=4, drop_last=True)

# Test using 60K Train Images
test_data = dset.MNIST('./dataset', train=True, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=4, drop_last=False)

## Inputs(datasets, noises) and Visualization Tool

In [7]:
# fixed random variables for test
c0 = torch.linspace(-1,1,10).view(-1,1).repeat(10,0)
c1 = torch.stack((c0, torch.zeros(1).expand_as(c0)),1).cuda()
c2 = torch.stack((torch.zeros(1).expand_as(c0), c0),1).cuda()
one_hot = torch.eye(10).repeat(1,1,10).view(100,10).cuda()
fix_noise = torch.Tensor(100, 62).uniform_(-1, 1).cuda()

# random noises sampling function
def _noise_sample(dis_c, con_c, noise, bs):
    idx = np.random.randint(10, size=bs)
    c = np.zeros((bs, 10))
    c[range(bs),idx] = 1.0
    dis_c.data.copy_(torch.Tensor(c))
    con_c.data.uniform_(-1.0, 1.0)
    noise.data.uniform_(-1.0, 1.0)
    z = torch.cat([noise, dis_c, con_c], 1).view(-1, 74, 1, 1)
    return z, idx

# Visdom
env_name = 'infoGAN'
vf = VisFunc(enval=env_name)

# Generated Image Folder
if not os.path.exists('tmp') : os.makedirs('tmp')

## Define Losses

![log_gaussian](https://user-images.githubusercontent.com/613623/30778123-7328abc0-a0cd-11e7-8998-7e4ef07cc25f.png)
https://user-images.githubusercontent.com/613623/30778123-7328abc0-a0cd-11e7-8998-7e4ef07cc25f.png

In [8]:
# Define Losses
class log_gaussian:
    def __call__(self, x, mu, var):
        logli = -0.5*(var.mul(2*np.pi)+1e-6).log() - (x-mu).pow(2).div(var.mul(2.0)+1e-6)
        return logli.sum(1).mean().mul(-1)

criterionD = nn.BCELoss()
criterionQ_dis = nn.CrossEntropyLoss()
criterionQ_con = log_gaussian()

## Functions for Classification Test

In [9]:
def set_mode(mode='train'):
    if mode == 'train' : 
        G.train()
        FE.train()
        D.train()
        Q.train()
    elif mode == 'eval' :
        G.eval()
        FE.eval()
        D.eval()
        Q.eval()
    else : raise BaseException('wrong mode') 
    
def test(num_class=10):
        '''
        1. make 2D table ( row : predictions, col : labels)
        2. calculate ACC using Hungarian Algorithm
        '''
        
        import torch.nn.functional as F
        from scipy.optimize import linear_sum_assignment
        
        set_mode('eval')
        
        LABEL = []
        PREDICT = []
        TABLE = [[0 for i in range(num_class)] for k in range(num_class)]
        total = 0
        for batch_idx, (images,labels) in enumerate(test_loader):
            real_x = Variable(images.cuda())
            fe_out = FE(real_x)
            q_logits, _, _ = Q(fe_out)
            q_softmax = F.softmax(q_logits) # redundant step
            q_index = q_softmax.max(1)[1]
            
            PREDICT.append(q_index.data)
            LABEL.append(labels)
            total += labels.size(0)

        LABEL = torch.cat(LABEL,0)
        PREDICT = torch.cat(PREDICT,0).cpu()
        for idx, label in enumerate(LABEL):
            TABLE[label][PREDICT[idx]] += 1

        TABLE = torch.FloatTensor(TABLE)
        row, col = linear_sum_assignment(-TABLE.numpy())
        acc = TABLE.numpy()[row,col].sum()/total

        print(TABLE)
        print('[TEST RESULT] : ACC : {:.4f}%'.format(acc*100))

        set_mode('train')

## Training

In [10]:
for epoch in range(100):
    if (epoch % 5) == 0 :  test(num_class=10)
    for num_iters, batch_data in enumerate(train_loader,0):

        # real part
        optimD.zero_grad()

        x, _ = batch_data
        real_x = Variable(x.cuda())
        label = Variable(torch.ones(batch_size).float().cuda(), requires_grad=False)

        fe_out1 = FE(real_x)
        probs_real = D(fe_out1)
        label.data.fill_(1)
        loss_real = criterionD(probs_real, label)
        loss_real.backward()

        # fake part
        dis_c = Variable(torch.FloatTensor(batch_size,10).cuda())
        con_c = Variable(torch.FloatTensor(batch_size,2).cuda())
        noise = Variable(torch.FloatTensor(batch_size,62).cuda())
        z, idx = _noise_sample(dis_c,con_c,noise,batch_size)

        fake_x = G(z)
        fe_out2 = FE(fake_x.detach())
        probs_fake = D(fe_out2)
        label.data.fill_(0)
        loss_fake = criterionD(probs_fake, label)
        loss_fake.backward()

        D_loss = loss_real + loss_fake
        optimD.step()

        # G and Q part
        optimG.zero_grad()

        fe_out = FE(fake_x)
        probs_fake = D(fe_out)
        label.data.fill_(1.0)
        reconstruct_loss = criterionD(probs_fake, label)

        q_logits, q_mu, q_var = Q(fe_out)
        class_ = torch.LongTensor(idx).cuda()
        target = Variable(class_)
        dis_loss = criterionQ_dis(q_logits, target)
        con_loss = criterionQ_con(con_c, q_mu, q_var)*0.1

        G_loss = reconstruct_loss + dis_loss + con_loss
        G_loss.backward()
        optimG.step()

        if num_iters % 50 == 0:
            print('Epoch:{}, Iter:{}, Dloss: {:.3f}, Gloss: {:.3f}, Preal: {:.3f}, Pfake: {:.3f}'.format(
                epoch, num_iters, D_loss.data[0],
                G_loss.data[0], probs_real.data.mean(), probs_fake.data.mean())
            )

            z = Variable(torch.cat([fix_noise, one_hot, c1], 1).view(-1, 74, 1, 1))
            x_save = G(z)
            title1 = '(C1)'+str(epoch)+'_'+str(num_iters)
            save_image(x_save.data, 'tmp/'+title1+'.png', nrow=10)
            vf.imshow_multi(x_save.data.cpu(), nrow=10, title=title1,factor=1)

            z = Variable(torch.cat([fix_noise, one_hot, c2], 1).view(-1, 74, 1, 1))
            x_save = G(z)
            title2 = '(C2)'+str(epoch)+'_'+str(num_iters)
            save_image(x_save.data, 'tmp/'+title2+'.png', nrow=10)
            vf.imshow_multi(x_save.data.cpu(), nrow=10, title=title2,factor=1)


    0  5923     0     0     0     0     0     0     0     0
    0  6742     0     0     0     0     0     0     0     0
    0  5958     0     0     0     0     0     0     0     0
    0  6131     0     0     0     0     0     0     0     0
    0  5842     0     0     0     0     0     0     0     0
    0  5421     0     0     0     0     0     0     0     0
    0  5918     0     0     0     0     0     0     0     0
    0  6265     0     0     0     0     0     0     0     0
    0  5851     0     0     0     0     0     0     0     0
    0  5949     0     0     0     0     0     0     0     0
[torch.FloatTensor of size 10x10]

[TEST RESULT] : ACC : 11.2367%


/home/wonkonge/anaconda3/envs/kongda/lib/python3.6/site-packages/torch/nn/functional.py:767: UserWarning: Using a target size (torch.Size([256])) that is different to the input size (torch.Size([256, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


Epoch:0, Iter:0, Dloss: 1.426, Gloss: 3.214, Preal: 0.543, Pfake: 0.510
Exception in user code:
------------------------------------------------------------


Traceback (most recent call last):
  File "/home/wonkonge/anaconda3/envs/kongda/lib/python3.6/site-packages/urllib3/connection.py", line 141, in _new_conn
    (self.host, self.port), self.timeout, **extra_kw)
  File "/home/wonkonge/anaconda3/envs/kongda/lib/python3.6/site-packages/urllib3/util/connection.py", line 83, in create_connection
    raise err
  File "/home/wonkonge/anaconda3/envs/kongda/lib/python3.6/site-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/wonkonge/anaconda3/envs/kongda/lib/python3.6/site-packages/urllib3/connectionpool.py", line 601, in urlopen
    chunked=chunked)
  File "/home/wonkonge/anaconda3/envs/kongda/lib/python3.6/site-packages/urllib3/connectionpool.py", line 357, in _make_request
    conn.request(method, url, **httplib_request_kw)
  File "/

Exception in user code:
------------------------------------------------------------
Epoch:1, Iter:0, Dloss: 1.142, Gloss: 1.299, Preal: 0.608, Pfake: 0.411
Exception in user code:
------------------------------------------------------------


...

Traceback (most recent call last):
  File "/home/wonkonge/anaconda3/envs/kongda/lib/python3.6/site-packages/urllib3/connection.py", line 141, in _new_conn
    (self.host, self.port), self.timeout, **extra_kw)
  File "/home/wonkonge/anaconda3/envs/kongda/lib/python3.6/site-packages/urllib3/util/connection.py", line 83, in create_connection
    raise err
  File "/home/wonkonge/anaconda3/envs/kongda/lib/python3.6/site-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/wonkonge/anaconda3/envs/kongda/lib/python3.6/site-packages/urllib3/connectionpool.py", line 601, in urlopen
    chunked=chunked)
  File "/home/wonkonge/anaconda3/envs/kongda/lib/python3.6/site-packages/urllib3/connectionpool.py", line 357, in _make_request
    conn.request(method, url, **httplib_request_kw)
  File "/

Epoch:17, Iter:0, Dloss: 1.159, Gloss: 0.874, Preal: 0.580, Pfake: 0.398
Exception in user code:
------------------------------------------------------------
Exception in user code:
------------------------------------------------------------


Traceback (most recent call last):
  File "/home/wonkonge/anaconda3/envs/kongda/lib/python3.6/site-packages/urllib3/connection.py", line 141, in _new_conn
    (self.host, self.port), self.timeout, **extra_kw)
  File "/home/wonkonge/anaconda3/envs/kongda/lib/python3.6/site-packages/urllib3/util/connection.py", line 83, in create_connection
    raise err
  File "/home/wonkonge/anaconda3/envs/kongda/lib/python3.6/site-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/wonkonge/anaconda3/envs/kongda/lib/python3.6/site-packages/urllib3/connectionpool.py", line 601, in urlopen
    chunked=chunked)
  File "/home/wonkonge/anaconda3/envs/kongda/lib/python3.6/site-packages/urllib3/connectionpool.py", line 357, in _make_request
    conn.request(method, url, **httplib_request_kw)
  File "/

Process Process-88:
  File "/home/wonkonge/anaconda3/envs/kongda/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
Process Process-86:
Process Process-87:
Process Process-85:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/wonkonge/anaconda3/envs/kongda/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/wonkonge/anaconda3/envs/kongda/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/wonkonge/anaconda3/envs/kongda/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/wonkonge/anaconda3/envs/kongda/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/wonkonge/anaconda3/envs/kongda/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*s

KeyboardInterrupt: 